# Resolve the Sidereal Time at Midnight of Site
- Define the site's longitude/latitude and the timezone.
- Figure out the dawn/dusk of the site over a range of days.
- Figure out the sidereal time at the midnight of these days at the site.
- The sidereal time now gives us the reference of the RA window that the telescope at the site may observe.

In [1]:
import requests
import json
import pprint
import pandas as pd
import numpy as np
import requests
import json
import datetime as dt
from datetime import datetime, timedelta, date
from zoneinfo import ZoneInfo
import pytz
from astral import LocationInfo
from astral.sun import sun

In [2]:
def create_midnight_datetime(timezone_str, date_val) -> datetime:
    """
    Conver the date_val to the midnight of the localized timezone. 
    """
    # Create a naive datetime object for midnight of the given date
    if(isinstance(date_val, str)):
        naive_datetime = datetime.strptime(date_val, '%Y-%m-%d').replace(hour=0, minute=0, second=0, microsecond=0)
    else:
        naive_datetime = datetime(date_val.year, date_val.month, date_val.day)
    
    # Convert naive datetime to a timezone-aware datetime using ZoneInfo
    timezone = ZoneInfo(timezone_str)
    localized_datetime = naive_datetime.replace(tzinfo=timezone)
    
    return localized_datetime

def to_hhmmss(localized_datetime:datetime, tz=pytz.utc) -> str:
    """
    Convert the localized_datetime to the corresponding hh:mm:ss of the timezon tz. 
    """
    return localized_datetime.astimezone(tz).strftime('%H:%M:%S')

# MST is -7 hours comparing with UTC
print(to_hhmmss(create_midnight_datetime("MST", date(2024, 2, 1))))
print(to_hhmmss(create_midnight_datetime("MST", "2024-02-01")))

07:00:00
07:00:00


In [3]:
def fetch_sidereal_time(d: str, longitude, latitude, reps, intv_mag=24, intv_unit="hours", time:str="00:00:00"):
    """
    Fetch the sidereal_time given the input.

    :param d: yyyy-mm-dd
    :param longitude: longitude
    :param latitude: latitude
    :param reps: The number of repetitions.
    :param intv_mag: The amount of interval.
    :param intv_unit: The unit of interval.
    :param time: the sun time of the corresponding sidereal time.
    """
    
    # URL of the API endpoint
    # url = "https://aa.usno.navy.mil/api/siderealtime?date=2020-03-04%20&coords=41.89,%2012.48&reps=3%20&intv_mag=12&intv_unit=hours%20&time=21:00:00"
    url = f"https://aa.usno.navy.mil/api/siderealtime?date={d}%20&coords={longitude},%20{latitude}&reps={reps}%20&intv_mag={intv_mag}&intv_unit={intv_unit}%20&time={time}"
    print(url)
    # Make a GET request to fetch the raw JSON data
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON data
        return response.json()
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        raise Error()

In [4]:
days=3

start_date = dt.date(2024, 7, 27)
end_date = start_date + dt.timedelta(days=days-1)

loc_meta = [
    {
        "site":"mt_lemmon",
        "long": 32.4420,
        "lat": -110.789,
        "tz": "MST",
    },
    {
        "site": "holmdel",
        "long": 40.3541,
        "lat":-74.184,
        "tz": "America/New_York",
    }
    ]
loc_meta = pd.DataFrame(loc_meta)

date_list = [(start_date + dt.timedelta(days=x)) for x in range((end_date - start_date).days + 1)]

In [5]:
data = []
for index, row in loc_meta[['site', 'long', 'lat']].iterrows():
    site, long, lat = row
    loc_info = LocationInfo("New York", "New York", "America/New_York", long, lat)
    data.append(pd.DataFrame([{"site": site, "date":d, **sun(loc_info.observer, date=d)} for d in date_list]))

df_sun_rise_set = pd.concat(data, axis=0)

df = pd.merge(loc_meta, df_sun_rise_set, left_on='site', right_on='site')
df['date'] = df['date'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [6]:
mt_lemmon = {"d": "2024-08-15", "longitude": 32.4420, "latitude":-110.789, "time": "07:00:00"}
holmdel = {"d": "2024-07-26", "longitude": 40.3541, "latitude":-74.184, "time": "04:00:00"}
site = holmdel

rows = []
for index, row in df[['site', 'long', 'lat', 'tz']].drop_duplicates().iterrows():
    site, long, lat, tz = row
    d = start_date.strftime("%Y-%m-%d")
    hhmmss = to_hhmmss(create_midnight_datetime(tz, d))
    raw_sidereal = fetch_sidereal_time(d, long, lat, reps=days, intv_mag=1, intv_unit="days", time=hhmmss)
    data_ele = raw_sidereal["properties"]["data"]
    df_sidereal = pd.DataFrame(data_ele)
    df_sidereal['site']=site
    df_sidereal['date'] = pd.to_datetime(df_sidereal[['year', 'month', 'day']])
    df_sidereal['date'] = df_sidereal['date'].dt.strftime('%Y-%m-%d')
    df_sidereal.drop(columns=['year', 'month', 'day'], inplace=True)
    rows.append(df_sidereal)
df_sidereal = pd.concat(rows, axis=0)

https://aa.usno.navy.mil/api/siderealtime?date=2024-07-27%20&coords=32.442,%20-110.789&reps=3%20&intv_mag=1&intv_unit=days%20&time=07:00:00
https://aa.usno.navy.mil/api/siderealtime?date=2024-07-27%20&coords=40.3541,%20-74.184&reps=3%20&intv_mag=1&intv_unit=days%20&time=04:00:00


In [7]:
df_complete=pd.merge(df, df_sidereal, on=['site', 'date'])
df_complete = df_complete[['site', 'date', 'long', 'lat', 'tz', 'dawn', 'sunrise', 
       'sunset', 'dawn', 'dusk', 'noon', 'eqofeq', 'gast', 'gmst', 'last', 'lmst', 'ut1time']]
df_complete

,site,date,long,lat,tz,dawn,sunrise,sunset,dawn,dusk,noon,eqofeq,gast,gmst,last,lmst,ut1time
0,mt_lemmon,2024-07-27,32.4420,-110.789,MST,2024-07-27 12:07:24.050186+00:00,2024-07-27 12:34:59.822836+00:00,2024-07-27 02:24:41.619108+00:00,2024-07-27 12:07:24.050186+00:00,2024-07-27 02:52:17.327641+00:00,2024-07-27 19:29:42+00:00,-0.1154,03:21:49.0243,03:21:49.1397,19:58:39.6643,19:58:39.7797,07:00:00.0
1,mt_lemmon,2024-07-28,32.4420,-110.789,MST,2024-07-28 12:08:07.775206+00:00,2024-07-28 12:35:39.650547+00:00,2024-07-28 02:23:59.611241+00:00,2024-07-28 12:08:07.775206+00:00,2024-07-28 02:51:31.409476+00:00,2024-07-28 19:29:41+00:00,-0.1170,03:25:45.5781,03:25:45.6951,20:02:36.2181,20:02:36.3351,07:00:00.0
2,mt_lemmon,2024-07-29,32.4420,-110.789,MST,2024-07-29 12:08:51.672200+00:00,2024-07-29 12:36:19.629591+00:00,2024-07-29 02:23:16.232861+00:00,2024-07-29 12:08:51.672200+00:00,2024-07-29 02:50:44.099418+00:00,2024-07-29 19:29:39+00:00,-0.1150,03:29:42.1354,03:29:42.2504,20:06:32.7754,20:06:32.8904,07:00:00.0
3,holmdel,2024-07-27,40.3541,-74.184,America/New_York,2024-07-27 09:18:53.215127+00:00,2024-07-27 09:50:26.284968+00:00,2024-07-27 00:16:29.783345+00:00,2024-07-27 09:18:53.215127+00:00,2024-07-27 00:48:02.513809+00:00,2024-07-27 17:03:17+00:00,-0.1150,00:21:19.4553,00:21:19.5703,19:24:35.2953,19:24:35.4103,04:00:00.0
4,holmdel,2024-07-28,40.3541,-74.184,America/New_York,2024-07-28 09:19:53.963939+00:00,2024-07-28 09:51:21.113597+00:00,2024-07-28 00:15:33.011222+00:00,2024-07-28 09:19:53.963939+00:00,2024-07-28 00:46:59.800859+00:00,2024-07-28 17:03:16+00:00,-0.1170,00:25:16.0086,00:25:16.1256,19:28:31.8486,19:28:31.9656,04:00:00.0
5,holmdel,2024-07-29,40.3541,-74.184,America/New_York,2024-07-29 09:20:55.130104+00:00,2024-07-29 09:52:16.340149+00:00,2024-07-29 00:14:34.621112+00:00,2024-07-29 09:20:55.130104+00:00,2024-07-29 00:45:55.449735+00:00,2024-07-29 17:03:14+00:00,-0.1155,00:29:12.5656,00:29:12.6810,19:32:28.4056,19:32:28.5210,04:00:00.0
